# Misbehavior Detection in Vehicular Networks with Federated Learning

## Imports

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import locale
import flwr as fl
import numpy as np
import pandas as pd
from abc import ABC
from metrics import *
import seaborn as sns
import tensorflow as tf
from itertools import cycle
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import shuffle
from typing import Optional, Tuple, Dict, Any
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_recall_curve, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, auc
from sklearn.metrics import PrecisionRecallDisplay, RocCurveDisplay, roc_curve
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_visible_devices(physical_devices[1:], 'GPU')
  logical_devices = tf.config.list_logical_devices('GPU')
  assert len(logical_devices) == len(physical_devices) - 1
except:
  pass

tf.get_logger().setLevel('ERROR')
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
locale._override_localeconv = {'thousands_sep': '.'}

from IPython.display import display, HTML
display(HTML("<style>.container { width:65% !important; }</style>"))

2024-10-16 21:27:31.255889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 21:27:31.451272: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 21:27:31.506211: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Support Functions

In [12]:
def load_veremi(csv_file: str, feature: str, label: str, delimiter=','):
    # Import VeReMi Dataset
    data = pd.read_csv(csv_file, delimiter=delimiter)

    # select columns
    columns = []
    for column in data.columns.values:
        if feature == 'feat1':
            if 'RSSI' in column:
                columns.append(column)
            elif 'distance' in column:
                columns.append(column)
        elif feature == 'feat2':
            if 'conformity' in column and '0' not in column:
                columns.append(column)
        elif feature == 'feat3':
            if 'RSSI' in column and '0' not in column:
                columns.append(column)
            elif 'distance' in column and '0' not in column:
                columns.append(column)
            elif 'conformity' in column and '0' not in column:
                columns.append(column)
        elif feature == 'feat4':
            if 'RSSI' in column:
                columns.append(column)
            elif 'aoa' in column:
                columns.append(column)
            elif 'distance' in column:
                columns.append(column)
            elif 'conformity' in column and '0' not in column:
                columns.append(column)
    columns.append('attack_type')

    # process target values
    if label == 'multiclass':
        data = data[columns]
    elif label == 'binary':
        pos_label = 1
        data = data[columns]
        data['attack_type'].loc[data['attack_type'] != 0] = pos_label
    else:
        pos_label = int(label.split("_")[1])
        data = data[columns]
        data = data.loc[(data['attack_type'] == 0) | (data['attack_type'] == pos_label)]

    data_normal = data.loc[data['attack_type'] == 0]
    data_atk = data.loc[data['attack_type'] != 0]
    # atk_size = int(data_atk.shape[0] * 1.5)
    atk_size = int(data_atk.shape[0])
    data = pd.concat([data_normal.sample(atk_size), data_atk])
    data = shuffle(data)

    dataset = data
    target = data[data.columns[-1:]]
    data = data[data.columns[0:-1]]

    # normalize data
    data = (data - data.mean()) / data.std()

    # label binarize one-hot style
    lb = preprocessing.LabelBinarizer()
    lb.fit(target)
    if label == 'multiclass':
        target = lb.transform(target)
    else:
        target = lb.transform(target)
        target = MultiLabelBinarizer().fit_transform(target)

    # Create training and test data
    train_data, test_data, train_labels, test_labels = train_test_split(
        data,
        target,
        train_size=Config.data_train_size,
        test_size=Config.data_test_size,
        # random_state=42
    )

    return train_data, test_data, train_labels, test_labels, lb, dataset

## Config Class

In [4]:
class Config:
    csv = "./VeReMi.csv"
    model_type = "mlp"
    label = "multiclass"
    feature = "feat4"
    batch_size = 128
    epochs = 50
    rounds = 80
    learning_rate = 3e-4 #1e-3
    min_available_clients = 2
    fraction_fit = 0.1
    early_stop_patience = 3
    early_stop_monitor = "loss"
    early_stop_min_delta = 1e-4
    early_stop_restore_best_weights = True
    data_train_size = 0.8
    data_test_size = 0.2
    output_path = f"results/{feature}/{label}/"
    performance_file = "performance.csv"
    weights_file = "model_weights.npz"

## Metrics

In [5]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras


def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

## VeReMi Base Class

In [6]:
class VeremiBase(ABC):
    def __init__(self, data_file: str, model_type: str, label: str, feature: str, activation: str = "softmax"):
        """ The Veremi Client Constructor
            :param model_type: Keras Model Type ('mlp' or 'lstm'
            :param label: Model label type ('binary', 'multiclass', 'atk_1', 'atk_2', 'atk_4', 'atk_8', 'atk_16')
            :param feature: Feature to evaluate ('feat1', 'feat2', 'feat3')
        """
        self.lb = None
        self.dataset = None
        self.train_data = None
        self.test_data = None
        self.train_labels = None
        self.test_labels = None
        self.model = None
        self.data_file = data_file
        self.label = label
        self.feature = feature
        self.model_type = model_type
        self.activation = activation

        self.load_veremi()
        self.create_model()

    def create_model(self):
        layer1, layer2, layer3, layer4, layer5, layer6, layer7, layer8, layer9, output = \
            None, None, None, None, None, None, None, None, None, None
        if self.model_type == 'mlp':
            layer1 = keras.layers.Input(shape=(self.train_data.shape[1],))
            layer2 = keras.layers.Dense(256, activation="relu")(layer1)
            layer3 = keras.layers.Dense(256, activation="relu")(layer2)
            layer4 = keras.layers.Dropout(0.5)(layer3)
            output = keras.layers.Dense(self.train_labels.shape[1], activation=self.activation)(layer4)
        else:
            pass

        # ML Model
        name = self.label + "-" + self.model_type + "-" + self.feature
        self.model = keras.Model(inputs=layer1, outputs=output, name=name)
        self.model.compile(
            loss=keras.losses.CategoricalCrossentropy(),
            optimizer=keras.optimizers.Adam(learning_rate=Config.learning_rate),
            metrics=[f1]
        )
        self.model.summary()

    def load_veremi(self):
        print("Loading dataset in " + self.__class__.__name__ + "...")
        self.train_data, self.test_data, self.train_labels, self.test_labels, self.lb, self.dataset = load_veremi(
            self.data_file,
            feature=self.feature,
            label=self.label
        )

## VeReMi Client Class

In [7]:
class VeremiClient(VeremiBase, fl.client.NumPyClient):
    def __init__(self, data_file: str, model_type: str, label: str, feature: str):
        VeremiBase.__init__(self, data_file, model_type, label, feature)
        self.history = None

    def get_parameters(self, config):
        return self.model.get_weights()
    
    def fit(self, parameters, config, verbose):
        print("Training...")
        early_stopping = keras.callbacks.EarlyStopping(
            monitor=Config.early_stop_monitor,
            patience=Config.early_stop_patience,
            min_delta=Config.early_stop_min_delta,
            restore_best_weights=Config.early_stop_restore_best_weights
        )
        if parameters is not None:
            self.model.set_weights(parameters)
        self.history = self.model.fit(
            self.train_data,
            self.train_labels,
            batch_size=config["batch_size"],
            epochs=config["epochs"],
            # callbacks=[early_stopping],
            validation_data=(self.test_data, self.test_labels),
            verbose=verbose,
        )
        result = {
            "f1_score:": float(self.history.history['f1'][-1]),
            "f1_val": float(self.history.history['val_f1'][-1]),
        }
        print("Finished!")
        return self.model.get_weights(), len(self.train_data), result

### Create Client

In [13]:
client = VeremiClient(Config.csv, Config.model_type, Config.label, Config.feature)

Loading dataset in VeremiClient...


ValueError: a must be greater than 0 unless no samples are taken

## VeReMi DataSet

In [ ]:
client.dataset

In [ ]:
client.train_data.describe()

In [ ]:
client.dataset.plot.scatter(x='aoa0', y='aoa1', c='attack_type', colormap='viridis')

In [ ]:
attack = len(client.dataset[client.dataset.attack_type != 0])
normal = len(client.dataset[client.dataset.attack_type == 0])
total = attack + normal
print('Attackers:\n    Total: {:,d}\n    Attack: {:,d} ({:.2f}% of total)\n'.format(
    total, attack, 100 * attack / total))

In [ ]:
atk_df = client.dataset.loc[client.dataset['attack_type'] != 0].sample(128)
normal_df = client.dataset.loc[client.dataset['attack_type'] == 0].sample(128)

# normalize data
columns = atk_df.columns[0:-1]
atk_df[columns] = (atk_df[columns] - atk_df[columns].mean()) / atk_df[columns].std()

columns = normal_df.columns[0:-1]
normal_df[columns] = (normal_df[columns] - normal_df[columns].mean()) / normal_df[columns].std()

atk_df

In [ ]:
atk_df.describe()

In [ ]:
normal_df

In [ ]:
# Plot
sns.jointplot(x=atk_df['RSSI0'], y=atk_df['distance0'], kind='hex')

In [ ]:
# Plot
sns.jointplot(x=atk_df['RSSI1'], y=atk_df['distance1'], kind='hex')

## Fit the MLP Model

### Load Model Params

In [ ]:
initial_parameters = None
file = Config.output_path + Config.weights_file
if os.path.exists(file):
    npzfile = np.load(file)
    params = [npzfile[x] for x in npzfile]
    params = fl.common.ndarrays_to_parameters(params)
    initial_parameters = fl.common.parameters_to_ndarrays(params)
    print("Setting model params...")
    client.model.set_weights(initial_parameters)

In [ ]:
results = client.fit(
    parameters=None, # initial_parameters,
    config={
        "batch_size": Config.batch_size,
        "epochs": Config.epochs
    },
    verbose=1
)

In [ ]:
train_f1_score = client.history.history['f1']
test_f1_score = client.history.history['val_f1']
train_loss = client.history.history['loss']
test_loss = client.history.history['val_loss']
epochs = range(1, Config.epochs + 1)
loss, num_examples, metrics = results

fig, (ax1, ax2) = plt.subplots(1,2)
fig.suptitle('Model Performance')
fig.set_figwidth(15)
fig.set_figheight(5)

ax1.plot(epochs, train_loss, '-g', label="Train Loss")
ax1.plot(epochs, test_loss, '-b', label="Test Loss")
ax1.legend()
ax1.set(xlabel='Epochs', ylabel='Loss')
ax1.set_title('Loss')

ax2.plot(epochs, train_f1_score, '-g', label="Train F1 Score")
ax2.plot(epochs, test_f1_score, '-b', label="Test F1 Score")
ax2.legend()
ax2.set(xlabel='Epochs', ylabel='F1 Score')
ax2.set_title('F1-Score')

plt.show()

In [ ]:
def plot_pr_roc_curves(probabilities: Any):
    n_classes = client.test_labels.shape[1]
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    precision = dict()
    recall = dict()
    pr_auc = dict()
        
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(client.test_labels[:, i], probabilities[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

        precision[i], recall[i], _ = precision_recall_curve(client.test_labels[:, i], probabilities[:, i])
        pr_auc[i] = auc(recall[i], precision[i])

    # First aggregate all false positive rates
    all_precision = np.unique(np.concatenate([precision[i] for i in range(n_classes)]))
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    mean_recall = np.zeros_like(all_precision)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
        mean_recall += np.interp(all_precision, precision[i], recall[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes
    mean_recall /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    precision["macro"] = all_precision
    recall["macro"] = mean_recall
    pr_auc["macro"] = auc(recall["macro"], precision["macro"])

    lw = 2
    
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.suptitle('PR and ROC Curves - Multiclass')
    fig.set_figwidth(15)
    fig.set_figheight(5)

    colors = cycle(["b", "g", "r", "c", "m", "y"])
    
    # ROC Curve
    ax1.plot(
        fpr["macro"],
        tpr["macro"],
        label="Macro Avg (area = {0:0.2f})".format(roc_auc["macro"]),
        color="navy",
        linestyle=":",
        linewidth=4,
        alpha=0.5,        
    )
    for i, color in zip(range(n_classes), colors):
        label_classes = int(client.lb.classes_[i])
        ax1.plot(
            fpr[i],
            tpr[i],
            color=color,
            lw=lw,
            label="Class {0} (area = {1:0.2f})".format(label_classes, roc_auc[i]),
            alpha=0.5
        )
    ax1.set_title(f"ROC Curve - {Config.feature} - {Config.label}")
    ax1.set_xlabel("False Positive Rate")
    ax1.set_ylabel("True Positive Rate")
    ax1.legend()

    # PR curve
    ax2.plot(
        precision["macro"],
        recall["macro"],
        label="Macro Avg (area = {0:0.2f})".format(pr_auc["macro"]),
        color="navy",
        linestyle=":",
        linewidth=4,
        alpha=0.5,
    )
    for i, color in zip(range(n_classes), colors):
        label_classes = int(client.lb.classes_[i])
        ax2.plot(
            precision[i],
            recall[i],
            color=color,
            lw=lw,
            label="Class {0} (area = {1:0.2f})".format(label_classes, pr_auc[i]),
            alpha=0.5
        )
    ax2.set_title(f"PR Curve - {Config.feature} - {Config.label}")    
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    
    plt.show()

In [ ]:
probabilities = client.model.predict(client.test_data)
inverse_target = client.lb.inverse_transform(client.test_labels)
prediction = None

if Config.label == 'multiclass':
    prediction = client.lb.inverse_transform(probabilities)
    # TODO: PLOT MULTICLASS 
    plot_pr_roc_curves(probabilities)
else:
    pos_label = 1 if Config.label == 'binary' else int(Config.label.split("_")[1])
    # Best threshold
    precision, recall, thresholds = precision_recall_curve(
        inverse_target,
        probabilities[:, 1],
        pos_label=pos_label
    )
    # convert to f score
    np.seterr(divide='ignore', invalid='ignore')
    fscore = (2 * precision * recall) / (precision + recall)
    np.nan_to_num(fscore, copy=False)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))
    print("-" * 70)
    
    prediction = np.where(np.array(probabilities[:, 1]) >= thresholds[ix], pos_label, 0)
    
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.suptitle('PR and ROC Curves')
    fig.set_figwidth(15)
    fig.set_figheight(5)

    # PR Curve
    PrecisionRecallDisplay.from_predictions(inverse_target, probabilities[:, 1], pos_label=pos_label, ax=ax1)
    no_skill = len(inverse_target[inverse_target == 1]) / len(inverse_target)
    ax1.plot([0, 1], [no_skill, no_skill], linestyle='--', color="grey", label='No Skill')
    ax1.scatter(recall[ix], precision[ix], marker='o', color='black', label='Best threshold')
    ax1.set_title(f"PR Curve - {Config.feature} - {Config.label}")
    ax1.legend()

    # ROC curve
    RocCurveDisplay.from_predictions(inverse_target, probabilities[:, 1], pos_label=pos_label, ax=ax2)
    ax2.plot([0, 1], [0, 1], color="grey", lw=1, linestyle="--")
    ax2.set_title(f"ROC Curve - {Config.feature} - {Config.label}")
    
    plt.show()

### Classification Report

In [ ]:
classlist = []
for cl in client.lb.classes_:
    classlist.append('class ' + str(int(cl)))

print(classification_report(inverse_target, prediction, target_names=classlist, digits=3, zero_division=0))
print("-" * 70)

### Confusion Matrix

In [ ]:
cm = confusion_matrix(inverse_target, prediction, labels=client.lb.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=client.lb.classes_)
disp.plot()
plt.title(f"Confusion Matrix - {Config.feature} - {Config.label}")
plt.show()

### Performance

In [ ]:
name = Config.label + "-" + Config.model_type + "-" + Config.feature
if Config.label == 'multiclass':
    prscore = precision_score(inverse_target, prediction, average='macro', zero_division=0)
    rcscore = recall_score(inverse_target, prediction, average='macro', zero_division=0)
    f1score = f1_score(inverse_target, prediction, average='macro', zero_division=0)
    accscore = accuracy_score(inverse_target, prediction)
else:
    prscore = precision_score(inverse_target, prediction, pos_label=pos_label, zero_division=0)
    rcscore = recall_score(inverse_target, prediction, pos_label=pos_label, zero_division=0)
    f1score = f1_score(inverse_target, prediction, pos_label=pos_label, zero_division=0)
    accscore = accuracy_score(inverse_target, prediction)
data_performance = {name: [prscore, rcscore, f1score, accscore]}
df_performance = pd.DataFrame.from_dict(data_performance, orient='index', columns=["precision", "recall", "f1score", "accuracy"])
df_performance